## Notebook Summary
This Notebook demo the usecase from quick start summary from the LangChain.\
The official URL:
https://python.langchain.com/docs/get_started/quickstart

## Import

In [9]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.documents import Document
from langchain.chains import create_retrieval_chain
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain.tools.retriever import create_retriever_tool
import langchain_util as util
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor




## Load environemnt variables

In [ ]:
# Load the .env file which contains the API keys
load_dotenv()

# Set the API keys as environment variables
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ['LANGCHAIN_TRACING_V2']= 'true'
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ["TAVILY_API_KEY"] = os.getenv('TAVILY_API_KEY')


# Optional, add tracing in LangSmith.
# This will help you visualize and debug the control flow
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_PROJECT"] = "Agentic_RAG_LANGGRAPH"

In [3]:
# You can uncomment the following lines to test the tools, but this is damn expensive.
# 2 runs costs aboout 4/1000 free token, where to upgrade you need to pay $100 for 15,000 tokens.
# for the cheapest option. So, be careful with the usage.
# Good luck!

# from langchain_community.tools.tavily_search import TavilySearchResults

# tool = TavilySearchResults()

# tool.invoke({"query": "What is tehr wheather in Toronto today?"})


## Baisc chain usage
No to useful, might just google for single question.

In [4]:
# llm = ChatOpenAI(model="gpt-3.5-turbo")

# llm.invoke("What is ChatPromptTemplate?")

## Langchain with prompt. The output is formatted with StrOutputParser
Similar to the above. But you can prompt the llm to format your outcome a bit more. Looks a bit nicer.

In [11]:
# prompt = ChatPromptTemplate.from_messages([
#     ("system", "You are world class technical documentation writer."),
#     ("user", "{input}")
# ])

# output_parser = StrOutputParser()
# chain = prompt | llm | output_parser

# chain.invoke({"input": "how can langsmith help with testing?"})



'Langsmith can help with testing in several ways:\n\n1. Automated Testing: Langsmith can generate test scripts to automate the testing process, making it faster and more efficient. These scripts can be tailored to specific requirements and scenarios, helping to identify bugs and issues in the software.\n\n2. Test Data Generation: Langsmith can generate test data that covers a wide range of input parameters and edge cases. This helps to ensure comprehensive test coverage and thorough validation of the software.\n\n3. Performance Testing: Langsmith can help with performance testing by generating load and stress test scenarios. This can help to identify potential bottlenecks and performance issues in the software.\n\n4. Regression Testing: Langsmith can generate scripts for regression testing to ensure that new code changes do not introduce new bugs or issues into the software.\n\n5. Continuous Integration: Langsmith can integrate with CI/CD pipelines to automate the testing process and p

## Load Retrive Message 
The RAG staart to get a little useful from here. This method uses the llm to generate answer base on the document you provided. I am using a sigle url, but one of the useful way to using this is to have relevant documents into a dir. Using the llm to handle entire dir might be a more practical usecase. Now the document can be in pdf, csv, md... the [llamaindex](https://www.llamaindex.ai/) allow you passing different format of files.

In [7]:

# # The llm
# llm = ChatOpenAI(temperature=0)

# # Build the vector database and set as retriever
# vector_db = util.build_db()
# retriever = vector_db.as_retriever()

# # set up the chain that takes a question and the retrieved documents and 
# # generates an answer.
# prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

# <context>
# {context}
# </context>

# Question: {input}""")

# # Takse a llm and temnplate.
# # There are different type of chains available in the langchain.
# # check out https://python.langchain.com/docs/modules/chains for detail
# document_chain = create_stuff_documents_chain(llm, prompt)

# retrieval_chain = create_retrieval_chain(retriever, document_chain)

# response = retrieval_chain.invoke({"input": "What are different LCEL chains??"})
                                
# print(response['answer'])


## Conversation Retrieval Chain
Passing conversation and prompt into the chain so we can have conversation with the llm, like the one online.\
We will need the pass the chat history into the chain. \
Perhaps that we need update the question and answer into history after retreie

In [10]:
# # First we need a prompt that we can pass into an LLM to generate this search query

# # The llm
# llm = ChatOpenAI(model="gpt-3.5-turbo")

# # Build the vector database from document and set as retriever
# vector_db = util.build_db()
# retriever = vector_db.as_retriever()

# # Engineer the prompt so it hold a conversation history
# query_gen_prompt = ChatPromptTemplate.from_messages([
#     MessagesPlaceholder(variable_name="chat_history"),
#     ("user", "{input}"),
#     ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
# ])

# # This retrieves chain link the llm, the database retriever and the prompt.
# # This retrieves chain will generate a search query based on the conversation history.
# query_gen_retriever_chain = create_history_aware_retriever(llm, retriever, query_gen_prompt)

# # Uncomment the following lines to test the chain.
# # chat_history = [HumanMessage(content="Is LCEL a great tool?"), AIMessage(content="Yes!")]

# # # Now we passing the chat history to the chain. 
# # # The chain will generate a search query based on the conversation history
# # # Hence. It is not the final answer to the question, but a search query that can be used to get the answer.
# # reponse = query_gen_retriever_chain.invoke({
# #     "chat_history": chat_history,
# #     "input": "Tell me how"
# # })


# # Now we combine the retriever chain that generate query with the document chain to generate the final answer.
# # The document chain will generate the final answer based on the search query generated by the retriever chain.
# final_prompt = ChatPromptTemplate.from_messages([
#     ("system", "Answer the user's questions based on the below context:\n\n{context}"),
#     MessagesPlaceholder(variable_name="chat_history"),
#     ("user", "{input}"),
# ])
# document_chain = create_stuff_documents_chain(llm, final_prompt)

# final_retrieval_chain = create_retrieval_chain(query_gen_retriever_chain, document_chain)

# # Uncomment the following lines to test the chain.
# chat_history = [HumanMessage(content="Is LCEL a great tool?"), AIMessage(content="Yes!")]
# final_retrieval_chain.invoke({
#     "chat_history": chat_history,
#     "input": "Tell me how"
# })

{'chat_history': [HumanMessage(content='Is LCEL a great tool?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content='Skip to main contentDocsUse casesIntegrationsGuidesAPIMorePeopleVersioningChangelogContributingTemplatesCookbooksTutorialsYouTube🦜️🔗LangSmithLangSmith DocsLangServe GitHubTemplates GitHubTemplates HubLangChain HubJS/TS Docs💬SearchGet startedIntroductionInstallationQuickstartSecurityLangChain Expression LanguageGet startedWhy use LCELInterfaceStreamingHow toCookbookLangChain Expression Language (LCEL)ModulesModel I/ORetrievalAgentsChainsMoreLangServeLangSmithLangGraphModulesChainsChainsChains refer to sequences of calls - whether to an LLM, a tool, or a\ndata preprocessing step. The primary supported way to do this is with\nLCEL.LCEL is great for constructing your own chains, but it’s also nice to\nhave chains that you can use off-the-shelf. There are two types of\noff-the-shelf chains that LangChain supports:Chains that are built wi

## Agent
The agent has abaility to search on web or reterive. 

In [11]:
# The llm
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# Build the vector database from document and set as retriever
vector_db = util.build_db()
retriever = vector_db.as_retriever()

# The tool that will be used to search for information about our document, passing a name and description. 
retriever_tool = create_retriever_tool(
    retriever,
    "chain_search",
    "Search for information about Chains. For any questions about Chain, you must use this tool!",
)

# The expensive search tool....
search = TavilySearchResults()

tools = [retriever_tool, search]

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# First test, ask a question that need apply a seach online
agent_executor.invoke({"input": "what is the weather in SF?"})




> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`


[{'url': 'https://forecast.weather.gov/zipcity.php?inputstring=San francisco,CA', 'content': 'NOAA National Weather Service National Weather Service. Toggle navigation. HOME; FORECAST . Local; Graphical; Aviation; Marine; Rivers and Lakes; Hurricanes; Severe Weather; Fire Weather; ... San Francisco CA 37.77°N 122.41°W (Elev. 131 ft) Last Update: 2:21 am PDT Mar 27, 2024. Forecast Valid: 7am PDT Mar 27, 2024-6pm PDT Apr 2, 2024 .'}, {'url': 'https://www.wunderground.com/forecast/us/ca/san-francisco', 'content': 'Get the latest 10-day weather forecast for San Francisco, CA, including temperature, precipitation, wind, pressure, humidity and more. See current conditions, sunrise and sunset times, moon phases and nearby weather stations.'}, {'url': 'https://www.accuweather.com/en/us/san-francisco/94103/weather-forecast/347629', 'content': 'Get the current and future

{'input': 'what is the weather in SF?',
 'output': 'I found some sources where you can check the weather in San Francisco:\n\n1. [NOAA National Weather Service](https://forecast.weather.gov/zipcity.php?inputstring=San%20francisco,CA): Provides local forecasts for San Francisco.\n2. [Weather Underground](https://www.wunderground.com/forecast/us/ca/san-francisco): Offers a 10-day weather forecast for San Francisco, including temperature, precipitation, wind, and more.\n3. [AccuWeather](https://www.accuweather.com/en/us/san-francisco/94103/weather-forecast/347629): Provides current and future weather conditions for San Francisco, including temperature, precipitation, wind, and air quality.\n4. [The Weather Channel](https://weather.com/weather/tenday/l/San%20Francisco%20CA%20USCA0987:1:US): Offers a 10-day weather forecast for San Francisco with hourly updates.\n5. [Hourly Weather Forecast](https://weather.com/weather/hourbyhour/l/USCA0987:1:US): Provides an hourly breakdown of the weather

In [13]:
# Second task. Ask a question that related to the document.
chat_history = [HumanMessage(content="Is LCEL chain a great tool?"), AIMessage(content="Yes!")]
agent_executor.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

# As the result, the agent uses different resources to get the answer.



> Entering new AgentExecutor chain...

Invoking: `chain_search` with `{'query': 'LCEL chain'}`


Skip to main contentDocsUse casesIntegrationsGuidesAPIMorePeopleVersioningChangelogContributingTemplatesCookbooksTutorialsYouTube🦜️🔗LangSmithLangSmith DocsLangServe GitHubTemplates GitHubTemplates HubLangChain HubJS/TS Docs💬SearchGet startedIntroductionInstallationQuickstartSecurityLangChain Expression LanguageGet startedWhy use LCELInterfaceStreamingHow toCookbookLangChain Expression Language (LCEL)ModulesModel I/ORetrievalAgentsChainsMoreLangServeLangSmithLangGraphModulesChainsChainsChains refer to sequences of calls - whether to an LLM, a tool, or a
data preprocessing step. The primary supported way to do this is with
LCEL.LCEL is great for constructing your own chains, but it’s also nice to
have chains that you can use off-the-shelf. There are two types of
off-the-shelf chains that LangChain supports:Chains that are built with LCEL. In this case, LangChain offers a
higher-level constr

{'chat_history': [HumanMessage(content='Is LCEL chain a great tool?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'output': 'The LCEL chain is a great tool for constructing your own chains and also for using off-the-shelf chains. It allows you to easily modify the internals of a chain, supports streaming, async, and batch operations out of the box, and provides observability at each step. It offers various chain constructors such as creating documents chains, SQL query chains, retriever chains, and more. Overall, the LCEL chain is a versatile and powerful tool for building and executing chains of operations.'}